# Introduction to Video Storyboarding with Amazon Nova Reel 1.1

Welcome to the third module of the Amazon Nova Reel Workshop! In this hands-on session, we'll explore the powerful video storyboarding capabilities of Amazon Nova Reel 1.1. This enhancement allows you to create videos up to 2 minutes long using either automated or manual storyboarding approaches.

## Use Case

OctankFashion wants to create more immersive marketing videos for their new summer t-shirt collection featuring palm tree designs. Traditional video production requires extensive planning, storyboarding, multiple shooting locations, and significant post-production work. With Nova Reel 1.1, OctankFashion can:

- Create longer form narrative videos that tell the story of how their products are designed, made, and enjoyed
- Transform existing product images into components of a larger storytelling video
- Reduce time and cost associated with traditional video production

## Workshop Objectives

By the end of this module, you will:

- Understand the difference between automated and manual storyboarding approaches
- Learn how to create longer narrative videos up to 2 minutes in length
- Experience how to build a video storyboard using both text-only and image-based approaches

## Features We'll Use

During this workshop module, we'll leverage the following features of Amazon Nova Reel 1.1:

- MULTI_SHOT_AUTOMATED: Create longer videos with a single comprehensive prompt
- MULTI_SHOT_MANUAL: Design each shot individually for greater creative control
- Image-based video generation within a multi-shot storyboard

Let's get started!


### Setting up your environment

First, we'll create an instance of the Bedrock Runtime client which we'll use to invoke the model.


In [ ]:
import boto3
import random
import json
import time
import base64
import video_gen_util
from IPython.display import display, Video

boto3.setup_default_session(region_name="us-east-1")
session = boto3.Session()
sts_client = session.client("sts")

bedrock_runtime = boto3.client("bedrock-runtime")

#### Setting up storage

Generating a video takes 1 minute or longer depending on the duration you've selected. To accommodate this execution time, the Bedrock Runtime introduces a new asynchronous invocation API. You start generating a video by calling the `start_async_invoke()` method on the Bedrock Runtime client. When the generation job completes, Bedrock automatically saves the generated video to an S3 bucket you specify.

Run the cell below to automatically create a new S3 bucket in your account which we will use as a destination for your videos. (If you already have an existing bucket you'd like to use, edit the `s3_destination_bucket` variable to use that bucket's ID)


In [ ]:
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]

# Replace this with an existing bucket name if you'd like.
bucket_name = f"nova-samples-{region}-{account_id}"

# Create the bucket
boto3.client("s3").create_bucket(Bucket=bucket_name)

### Example 1: Automated Multi-Shot Video

OctankFashion wants to create a promotional video featuring lifestyle imagery reflecting the inspiration for their palm tree t-shirt collection. With Nova Reel 1.1's automated multi-shot feature, they can describe the entire narrative in a single prompt and let the model determine the best way to segment and sequence the shots.

The **MULTI_SHOT_AUTOMATED** task type accepts:

- A single text prompt up to 4,000 characters
- Duration parameter (multiple of 6 seconds, between 12-120 seconds)
- No input images are supported in this mode

Let's generate a 30-second automated multi-shot video showcasing OctankFashion's palm tree t-shirts:


In [ ]:
# Define the main input parameters
text_prompt = "A cinematic video that showcases the beauty and calm of a tropical island. Start with a closeup of a shell in the sand near the surf. Transition to handheld shot (shakey) of a simple, welcoming hut at the edge of a jungle of palm trees bordering the sand, slowy approaching the hut's dark entrance. End with a drone shot of the beautiful isolated tropical island the hut is on."

duration_seconds = 18  # Must be a multiple of 6 in range [12, 120]
seed = 1016  # Setting a specific seed for reproducibility

model_input = {
    "taskType": "MULTI_SHOT_AUTOMATED",
    "multiShotAutomatedParams": {"text": text_prompt},
    "videoGenerationConfig": {
        "durationSeconds": duration_seconds,
        "fps": 24,  # Must be 24
        "dimension": "1280x720",  # Must be "1280x720"
        "seed": seed,
    },
}

# Start the asynchronous video generation job
invocation = bedrock_runtime.start_async_invoke(
    modelId="amazon.nova-reel-v1:1",
    modelInput=model_input,
    outputDataConfig={"s3OutputDataConfig": {"s3Uri": f"s3://{bucket_name}"}},
)

# Store the invocation ARN for later status checking
invocation_arn = invocation["invocationArn"]
job_id = invocation_arn.split("/")[-1]
s3_location = f"s3://{bucket_name}/{job_id}"

# Print the response
print("\nResponse:")
print(json.dumps(invocation, indent=2, default=str))
print(f"\nVideo will be available at: {s3_location}/output.mp4 when complete")

# Save invocation details for reference
video_gen_util.save_invocation_info(invocation, model_input)

#### Checking the Status of Your Video Generation

The following code will check the current status of the video generation job. Run this code to see how it works, then continue on. (We'll be downloading completed jobs automatically later in this notebook.)


In [ ]:
response = bedrock_runtime.get_async_invoke(invocationArn=invocation_arn)
status = response["status"]
print(f"Status: {status}")
print("\nFull response:")
print(json.dumps(response, indent=2, default=str))

### Example 2: Manual Multi-Shot Video (Text-Only)

For greater creative control, OctankFashion can use Nova Reel 1.1's manual storyboarding feature. This allows you to precisely define each shot in your video with its own unique prompt. The **MULTI_SHOT_MANUAL** task type lets you:

- Create up to 20 individual shots (each 6 seconds long, resulting in up to 2 minutes of video)
- Define each shot with its own text prompt (up to 512 characters)
- Optionally include a reference image for any shot

Let's create a manual storyboard that tells the story of OctankFashion's palm tree T-shirt from concept to finished product:


In [ ]:
# Define each shot in the storyboard
shots = [
    {
        "text": "Aerial drone shot of a tropical beach with palm trees swaying in the breeze. Golden morning light bathes the scene. Professional, cinematic quality with rich colors."
    },
    {
        "text": "Close-up of design sketches of palm tree patterns spread on a wooden table, a pencil rolls slowly across the paper. Soft, natural lighting from a nearby window."
    },
    {
        "text": "Wide shot of a clothing workshop with fabric being cut by machine into white t-shirt patterns. Clean, organized space with muted colors and professional equipment in operation."
    },
    {
        "text": "Slow motion macro shot of a finished white t-shirt with the palm tree graphic being illuminated by changing light, showing the texture and print quality in detail."
    },
    {
        "text": "Smooth dolly shot of the finished white palm tree graphic t-shirt displayed on a minimalist mannequin torso against a white background, slowly rotating to show all angles."
    },
]

# Setting seed for reproducibility
seed = 230

model_input = {
    "taskType": "MULTI_SHOT_MANUAL",
    "multiShotManualParams": {"shots": shots},
    "videoGenerationConfig": {
        "fps": 24,  # Must be 24
        "dimension": "1280x720",  # Must be "1280x720"
        "seed": seed,
    },
}

# Start the asynchronous video generation job
invocation = bedrock_runtime.start_async_invoke(
    modelId="amazon.nova-reel-v1:1",
    modelInput=model_input,
    outputDataConfig={"s3OutputDataConfig": {"s3Uri": f"s3://{bucket_name}"}},
)

# Store the invocation ARN for later status checking
invocation_arn_manual = invocation["invocationArn"]
job_id = invocation_arn_manual.split("/")[-1]
s3_location = f"s3://{bucket_name}/{job_id}"

# Print the response
print("\nResponse:")
print(json.dumps(invocation, indent=2, default=str))
print(f"\nVideo will be available at: {s3_location}/output.mp4 when complete")

# Save invocation details for reference
video_gen_util.save_invocation_info(invocation, model_input)

### Example 3: Manual Multi-Shot Video with Images

For the most precise control, OctankFashion can use their actual product images as the foundation for specific shots in their promotional video. This approach is ideal when you want to ensure brand consistency and product accuracy in the final video.

The image-based approach allows you to:

- Provide a reference image for any or all shots in your storyboard
- Add camera movement and life to static product photos
- Maintain perfect product representation while adding motion and context

Here are the images we'll be using:

<div style="display: flex; max-width:100%; gap:10px">
  <div style="flex: 1; display: flex; flex-direction: column; align-items: center">
    <img src="data/ref_img_1.png">
  </div>
  <div style="flex: 1; display: flex; flex-direction: column; align-items: center">
    <img src="data/ref_img_2.png">
  </div>
  <div style="flex: 1; display: flex; flex-direction: column; align-items: center">
    <img src="data/ref_img_3.png">
  </div>
</div>

Let's create a function to help us convert images to base64 format:


In [ ]:
def image_to_base64(image_path: str):
    """
    Helper function which converts an image file to a base64 encoded string.
    """
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read())
        return encoded_string.decode("utf-8")

Now, let's create a storyboard using OctankFashion's actual product images:


In [ ]:
# Define shots using both text and images
shots = [
    {
        "text": "Slow tracking shot moving across the retail display, focusing on the palm tree t-shirt. Static clothing display with soft store lighting. Camera gradually moves from left to right, revealing more of the store environment in the background.",
        "image": {
            "format": "png",
            "source": {"bytes": image_to_base64("data/ref_img_1.png")},
        },
    },
    {
        "text": "Gentle breeze moves palm tree branches and rustles the t-shirt fabric. Waves roll in the distance, beach sand shifts slightly with the wind. Natural tropical sunlight creates perfect lighting on the shirt design.",
        "image": {
            "format": "png",
            "source": {"bytes": image_to_base64("data/ref_img_2.png")},
        },
    },
    {
        "text": "Slow dolly back shot revealing the entire poolside scene. Water ripples gently in the pool, reflective light plays across the t-shirt fabric. A tranquil resort atmosphere with subtle ambient movement.",
        "image": {
            "format": "png",
            "source": {"bytes": image_to_base64("data/ref_img_3.png")},
        },
    },
]

# Setting seed for reproducibility
seed = 42

model_input = {
    "taskType": "MULTI_SHOT_MANUAL",
    "multiShotManualParams": {"shots": shots},
    "videoGenerationConfig": {
        "fps": 24,  # Must be 24
        "dimension": "1280x720",  # Must be "1280x720"
        "seed": seed,
    },
}

# Start the asynchronous video generation job
invocation = bedrock_runtime.start_async_invoke(
    modelId="amazon.nova-reel-v1:1",
    modelInput=model_input,
    outputDataConfig={"s3OutputDataConfig": {"s3Uri": f"s3://{bucket_name}"}},
)

# Store the invocation ARN for later status checking
invocation_arn_image = invocation["invocationArn"]
job_id = invocation_arn_image.split("/")[-1]
s3_location = f"s3://{bucket_name}/{job_id}"

# Print the response
print("\nResponse:")
print(json.dumps(invocation, indent=2, default=str))
print(f"\nVideo will be available at: {s3_location}/output.mp4 when complete")

# Save invocation details for reference
video_gen_util.save_invocation_info(invocation, model_input)

##### Download and view the generated videos

We've provided a set of utility functions in the `video_gen_util.py` script. One of these functions provides one solution to automatically downloading previously completed jobs and monitoring in-progress jobs. Finished jobs will be automatically downloaded to the "output" folder.


In [ ]:
from datetime import datetime, timedelta, timezone

# Download and monitor videos from the past N hours
duration_hours = 2

from_submit_time = datetime.now(timezone.utc) - timedelta(hours=duration_hours)

local_file_paths = video_gen_util.monitor_and_download_videos("output", submit_time_after=from_submit_time)

for video in local_file_paths:
    display(Video(video, embed=True, html_attributes="controls loop autoplay"))


### Additional Options: Using S3 Images

For larger productions, OctankFashion might want to store their image assets in S3 directly. Nova Reel 1.1 also supports referencing S3 images in your shot definitions. Instead of providing base64 encoded images, you can reference S3 locations:


##### Example structure for S3 image reference (not run in this notebook)

```python
"image": {
    "format": "png",  # Must be "png" or "jpeg"
    "source": {
        "s3Location": {
            "uri": "s3://your-bucket-name/path/to/image.png",
            "bucketOwner": "optional-bucket-owner-id"
        }
    },
}
```


This approach is particularly useful when building automated workflows or when working with a large image library.


## Take Away

Amazon Nova Reel 1.1's storyboarding features give OctankFashion unprecedented control over their marketing videos. By combining automated multi-shot generation, manual storyboarding, and image-based video creation, they can create compelling narrative videos that showcase their products from concept to customer.

In this workshop, you've learned how to generate long-form video content for fashion marketing using both automated and manual storyboarding approaches. With these techniques, OctankFashion can dramatically expand their video marketing capabilities while reducing production time and costs.
